In [2]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import OneClassSVM
from sklearn.neighbors import LocalOutlierFactor
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from keras.layers import Input, Dense
from keras.models import Model
from sklearn.metrics import precision_score, recall_score, f1_score
from sklearn import svm
import warnings
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import DBSCAN
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.feature_selection import SelectKBest
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

# Process benign only dataset

In [3]:
import pandas as pd
import ast

# Load the dataset
data = pd.read_csv('/kaggle/input/benignonly/benignonly.csv')

# Drop columns with constant values
columns_to_drop = ['ip_version', 'vlan_id', 'tunnel_id']

# Fill in missing values with zeros
data['splt_direction'] = data['splt_direction'].fillna('[0, 0, 0]')
data['splt_ps'] = data['splt_ps'].fillna('[0, 0, 0]')
data['splt_piat_ms'] = data['splt_piat_ms'].fillna('[0, 0, 0]')

# Split the 'splt_direction', 'splt_ps', and 'splt_piat_ms' columns into three separate columns
splt_direction = pd.DataFrame(data['splt_direction'].apply(lambda x: ast.literal_eval(x) if isinstance(x, str) else [0, 0, 0]).tolist(), columns=['splt_direction_1', 'splt_direction_2', 'splt_direction_3'])
splt_ps = pd.DataFrame(data['splt_ps'].apply(lambda x: ast.literal_eval(x) if isinstance(x, str) else [0, 0, 0]).tolist(), columns=['splt_ps_1', 'splt_ps_2', 'splt_ps_3'])
splt_piat_ms = pd.DataFrame(data['splt_piat_ms'].apply(lambda x: ast.literal_eval(x) if isinstance(x, str) else [0, 0, 0]).tolist(), columns=['splt_piat_ms_1', 'splt_piat_ms_2', 'splt_piat_ms_3'])
data = pd.concat([data, splt_direction, splt_ps, splt_piat_ms], axis=1)
data = data.drop(['splt_direction', 'splt_ps', 'splt_piat_ms'], axis=1)
data = data.drop(columns_to_drop, axis=1)

data.head()

,expiration_id,src_port,dst_port,protocol,bidirectional_first_seen_ms,bidirectional_last_seen_ms,bidirectional_duration_ms,bidirectional_packets,bidirectional_bytes,src2dst_first_seen_ms,...,Label,splt_direction_1,splt_direction_2,splt_direction_3,splt_ps_1,splt_ps_2,splt_ps_3,splt_piat_ms_1,splt_piat_ms_2,splt_piat_ms_3
0,0,62512,7777,17,1.680000e+12,1.680000e+12,899,8,13520,1.680000e+12,...,benign,0,0,0,4600,576,1800,0,12,4
1,1,62512,7777,17,1.680000e+12,1.680000e+12,3192,61,58045,1.680000e+12,...,benign,0,0,0,576,576,576,0,22,6
2,0,8889,6525,17,1.680000e+12,1.680000e+12,3419,21,5782,1.680000e+12,...,benign,0,0,0,127,127,287,0,363,240
3,1,62512,7777,17,1.680000e+12,1.680000e+12,4869,79,90302,1.680000e+12,...,benign,0,0,0,576,576,576,0,744,244
4,1,8889,6525,17,1.680000e+12,1.680000e+12,4636,23,15064,1.680000e+12,...,benign,0,0,0,576,576,836,0,1393,101


# Process malicious only dataset

In [4]:
# Load the dataset
mal_data = pd.read_csv('/kaggle/input/maliciousonly/malicious_data1.csv')

# Drop the columns you don't need
columns_to_drop = ['ip_version', 'vlan_id', 'tunnel_id']


# Fill in missing values with zeros
mal_data['splt_direction'] = mal_data['splt_direction'].fillna('[0, 0, 0]')
mal_data['splt_ps'] = mal_data['splt_ps'].fillna('[0, 0, 0]')
mal_data['splt_piat_ms'] = mal_data['splt_piat_ms'].fillna('[0, 0, 0]')

# Split the 'splt_direction', 'splt_ps', and 'splt_piat_ms' columns into three separate columns
splt_direction = pd.DataFrame(mal_data['splt_direction'].apply(eval).tolist(), columns=['splt_direction_1', 'splt_direction_2', 'splt_direction_3'])
splt_ps = pd.DataFrame(mal_data['splt_ps'].apply(eval).tolist(), columns=['splt_ps_1', 'splt_ps_2', 'splt_ps_3'])
splt_piat_ms = pd.DataFrame(mal_data['splt_piat_ms'].apply(eval).tolist(), columns=['splt_piat_ms_1', 'splt_piat_ms_2', 'splt_piat_ms_3'])
mal_data = pd.concat([mal_data, splt_direction, splt_ps, splt_piat_ms], axis=1)
mal_data = mal_data.drop(['splt_direction', 'splt_ps', 'splt_piat_ms'], axis=1)
mal_data = mal_data.drop(columns_to_drop, axis=1)

mal_data.head()


,expiration_id,src_port,dst_port,protocol,bidirectional_first_seen_ms,bidirectional_last_seen_ms,bidirectional_duration_ms,bidirectional_packets,bidirectional_bytes,src2dst_first_seen_ms,...,Label,splt_direction_1,splt_direction_2,splt_direction_3,splt_ps_1,splt_ps_2,splt_ps_3,splt_piat_ms_1,splt_piat_ms_2,splt_piat_ms_3
0,1,62512,7777,17,1.680000e+12,1.680000e+12,4980,61,52056,1.680000e+12,...,hijack,0,0,0,576,576,576,0,194,73
1,1,6525,8889,17,1.680000e+12,1.680000e+12,4923,69,24961,1.680000e+12,...,hijack,0,0,0,184,114,114,0,35,22
2,1,62512,7777,17,1.680000e+12,1.680000e+12,4997,83,90004,1.680000e+12,...,hijack,0,0,0,576,576,1944,0,1141,175
3,1,8889,6525,17,1.680000e+12,1.680000e+12,4882,28,6747,1.680000e+12,...,hijack,0,1,1,200,114,114,0,168,25
4,1,62512,7777,17,1.680000e+12,1.680000e+12,4975,187,167412,1.680000e+12,...,hijack,0,0,0,576,576,576,0,30,4


# Stage 1 , novelty deetction model

In [6]:
from sklearn.neighbors import LocalOutlierFactor
from sklearn.metrics import classification_report
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import f_classif

warnings.filterwarnings("ignore", category=RuntimeWarning)
warnings.filterwarnings("ignore", category=UserWarning)

# Update the Label column to binary values
data['Label'] = 0
mal_data['Label'] = 1

# Split the benign data into train and test sets
X_train_benign, X_test_benign = train_test_split(data, test_size=0.2, random_state=42)

# Combine the benign test set with the malicious data for testing
X_test = pd.concat([X_test_benign, mal_data], ignore_index=True)

# Separate the features (X_train) and the labels (y_train) for training
X_train = X_train_benign.drop("Label", axis=1)
y_train = X_train_benign["Label"]

# Separate the labels (y_test) for testing
y_test = X_test["Label"]

# Remove the label column from the test data
X_test = X_test.drop("Label", axis=1)

# Apply feature selection
selector = SelectKBest(score_func=f_classif, k=30)  # Adjust the value of k as needed
X_train_selected = selector.fit_transform(X_train, y_train)
X_test_selected = selector.transform(X_test)
selected_features = X_train.columns[selector.get_support(indices=True)]
print(selected_features)

# Normalize your data
scaler = StandardScaler()
X_train_normalized = scaler.fit_transform(X_train_selected)
X_test_normalized = scaler.transform(X_test_selected)

# LOF MODEL

# Train the local outlier factor (LOF) model for novelty detection
lof_novelty = LocalOutlierFactor(n_neighbors=5, novelty=True).fit(X_train_normalized, y_train)

prediction_novelty = lof_novelty.predict(X_test_normalized)

scores = lof_novelty.decision_function(X_test_normalized)

prediction_novelty = [1 if i == -1 else 0 for i in prediction_novelty]

lof_accuracy = np.mean(prediction_novelty == y_test)
print(f"outlier factor Accuracy: {lof_accuracy}")

# Check the model performance
print(classification_report(y_test, prediction_novelty))


Index(['bidirectional_psh_packets', 'bidirectional_rst_packets',
       'bidirectional_fin_packets', 'src2dst_syn_packets',
       'src2dst_cwr_packets', 'src2dst_ece_packets', 'src2dst_urg_packets',
       'src2dst_ack_packets', 'src2dst_psh_packets', 'src2dst_rst_packets',
       'src2dst_fin_packets', 'dst2src_syn_packets', 'dst2src_cwr_packets',
       'dst2src_ece_packets', 'dst2src_urg_packets', 'dst2src_ack_packets',
       'dst2src_psh_packets', 'dst2src_rst_packets', 'dst2src_fin_packets',
       'application_is_guessed', 'application_confidence', 'splt_direction_1',
       'splt_direction_2', 'splt_direction_3', 'splt_ps_1', 'splt_ps_2',
       'splt_ps_3', 'splt_piat_ms_1', 'splt_piat_ms_2', 'splt_piat_ms_3'],
      dtype='object')
outlier factor Accuracy: 0.8265933354901327
              precision    recall  f1-score   support

           0       0.62      0.74      0.68       755
           1       0.91      0.85      0.88      2336

    accuracy                           

# OneClassSVM 

In [8]:
from sklearn.svm import OneClassSVM
from sklearn.metrics import classification_report
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import f_classif

warnings.filterwarnings("ignore", category=RuntimeWarning)
warnings.filterwarnings("ignore", category=UserWarning)

# Split the benign data into train and test sets
X_train_benign, X_test_benign = train_test_split(data, test_size=0.2, random_state=42)

# Combine the benign test set with the malicious data for testing
X_test = pd.concat([X_test_benign, mal_data], ignore_index=True)

# Separate the features (X_train) and the labels (y_train) for training
X_train = X_train_benign.drop("Label", axis=1)
y_train = X_train_benign["Label"]

# Separate the labels (y_test) for testing
y_test = X_test["Label"]

# Remove the label column from the test data
X_test = X_test.drop("Label", axis=1)

# Apply feature selection
selector = SelectKBest(score_func=f_classif, k=30)  # Adjust the value of k as needed
X_train_selected = selector.fit_transform(X_train, y_train)
X_test_selected = selector.transform(X_test)
selected_features = X_train.columns[selector.get_support(indices=True)]
print(selected_features)

    
# Normalize your data
scaler = StandardScaler()
X_train_normalized = scaler.fit_transform(X_train_selected)
X_test_normalized = scaler.transform(X_test_selected)

# One-Class SVM MODEL 

# Train the one-class SVM model for novelty detection
svm_novelty = OneClassSVM(kernel='rbf', nu=0.05).fit(X_train_normalized)

prediction_novelty = svm_novelty.predict(X_test_normalized)
prediction_novelty = [1 if i == -1 else 0 for i in prediction_novelty]

svm_accuracy = np.mean(prediction_novelty == y_test)
print(f"One-Class SVM Accuracy: {svm_accuracy}")

# Check the model performance
print(classification_report(y_test, prediction_novelty))


Index(['bidirectional_psh_packets', 'bidirectional_rst_packets',
       'bidirectional_fin_packets', 'src2dst_syn_packets',
       'src2dst_cwr_packets', 'src2dst_ece_packets', 'src2dst_urg_packets',
       'src2dst_ack_packets', 'src2dst_psh_packets', 'src2dst_rst_packets',
       'src2dst_fin_packets', 'dst2src_syn_packets', 'dst2src_cwr_packets',
       'dst2src_ece_packets', 'dst2src_urg_packets', 'dst2src_ack_packets',
       'dst2src_psh_packets', 'dst2src_rst_packets', 'dst2src_fin_packets',
       'application_is_guessed', 'application_confidence', 'splt_direction_1',
       'splt_direction_2', 'splt_direction_3', 'splt_ps_1', 'splt_ps_2',
       'splt_ps_3', 'splt_piat_ms_1', 'splt_piat_ms_2', 'splt_piat_ms_3'],
      dtype='object')
One-Class SVM Accuracy: 0.8434163701067615
              precision    recall  f1-score   support

           0       0.62      0.93      0.74       755
           1       0.97      0.82      0.89      2336

    accuracy                           0

# Export model 1

In [9]:
import pickle
# Save the model as a pickle file
with open('/kaggle/working/phase_1_model.pkl', 'wb') as f:
    pickle.dump(svm_novelty, f)

In [10]:

# Save the scaler
with open('standard_scaler1.pkl', 'wb') as file:
    pickle.dump(scaler, file)

# stage 2 classification

 **Process Dataset**

In [11]:
import pandas as pd
from sklearn.feature_selection import SelectKBest, f_classif
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score


# Load the dataset
data = pd.read_csv('/kaggle/input/maliciousonly/malicious_data1.csv')


# Drop the columns you don't need
columns_to_drop = ['vlan_id', 'tunnel_id', 'bidirectional_cwr_packets', 'bidirectional_ece_packets', 'bidirectional_urg_packets', 'bidirectional_psh_packets', 'src2dst_cwr_packets', 'src2dst_ece_packets', 'src2dst_urg_packets', 'src2dst_psh_packets', 'dst2src_cwr_packets', 'dst2src_ece_packets', 'dst2src_urg_packets', 'dst2src_psh_packets', 'application_is_guessed',  'splt_ps_3']

# Fill in missing values with zeros
data['splt_direction'] = data['splt_direction'].fillna('[0, 0, 0]')
data['splt_ps'] = data['splt_ps'].fillna('[0, 0, 0]')
data['splt_piat_ms'] = data['splt_piat_ms'].fillna('[0, 0, 0]')

# Split the 'splt_direction', 'splt_ps', and 'splt_piat_ms' columns into three separate columns
splt_direction = pd.DataFrame(data['splt_direction'].apply(eval).tolist(), columns=['splt_direction_1', 'splt_direction_2', 'splt_direction_3'])
splt_ps = pd.DataFrame(data['splt_ps'].apply(eval).tolist(), columns=['splt_ps_1', 'splt_ps_2', 'splt_ps_3'])
splt_piat_ms = pd.DataFrame(data['splt_piat_ms'].apply(eval).tolist(), columns=['splt_piat_ms_1', 'splt_piat_ms_2', 'splt_piat_ms_3'])
data = pd.concat([data, splt_direction, splt_ps, splt_piat_ms], axis=1)
data = data.drop(['splt_direction', 'splt_ps', 'splt_piat_ms'], axis=1)
data = data.drop(columns_to_drop, axis=1)
data.head()

,expiration_id,src_port,dst_port,protocol,ip_version,bidirectional_first_seen_ms,bidirectional_last_seen_ms,bidirectional_duration_ms,bidirectional_packets,bidirectional_bytes,...,application_confidence,Label,splt_direction_1,splt_direction_2,splt_direction_3,splt_ps_1,splt_ps_2,splt_piat_ms_1,splt_piat_ms_2,splt_piat_ms_3
0,1,62512,7777,17,4,1.680000e+12,1.680000e+12,4980,61,52056,...,0,hijack,0,0,0,576,576,0,194,73
1,1,6525,8889,17,4,1.680000e+12,1.680000e+12,4923,69,24961,...,0,hijack,0,0,0,184,114,0,35,22
2,1,62512,7777,17,4,1.680000e+12,1.680000e+12,4997,83,90004,...,0,hijack,0,0,0,576,576,0,1141,175
3,1,8889,6525,17,4,1.680000e+12,1.680000e+12,4882,28,6747,...,0,hijack,0,1,1,200,114,0,168,25
4,1,62512,7777,17,4,1.680000e+12,1.680000e+12,4975,187,167412,...,0,hijack,0,0,0,576,576,0,30,4


**Building Model II**

In [12]:
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.feature_selection import SelectKBest, f_classif
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import GaussianNB
import pandas as pd

warnings.filterwarnings("ignore", category=RuntimeWarning)
warnings.filterwarnings("ignore", category=UserWarning)

# Split the dataset into features (X) and labels (y)
X = data.drop('Label', axis=1)
y = data['Label']
le = LabelEncoder()
y = le.fit_transform(y)


# Perform feature selection to select the 10 best features
selector = SelectKBest(f_classif, k=20)
X_new = selector.fit_transform(X, y)
selected_features = X.columns[selector.get_support(indices=True)]
print(selected_features)

# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_new, y, test_size=0.2, random_state=42)

# Normalize the features using StandardScaler
scaler1 = StandardScaler()
X_scaled = scaler1.fit_transform(X_train)
X_test_scaled = scaler1.transform(X_test)

# Train and test a Random Forest model
rf = RandomForestClassifier()
rf.fit(X_scaled, y_train)
rf_pred = rf.predict(X_test_scaled)
rf_acc = accuracy_score(y_test, rf_pred)
rf_precision = precision_score(y_test, rf_pred, average='macro')
rf_recall = recall_score(y_test, rf_pred, average='macro')
rf_f1 = f1_score(y_test, rf_pred, average='macro')
print('Random Forest Accuracy:', rf_acc)
print('Random Forest Precision:', rf_precision)
print('Random Forest Recall:', rf_recall)
print('Random Forest F1 Score:', rf_f1)

# Train and test an XGBoost model
xgb = XGBClassifier()
xgb.fit(X_train, y_train)
xgb_pred = xgb.predict(X_test)
xgb_acc = accuracy_score(y_test, xgb_pred)
xgb_precision = precision_score(y_test, xgb_pred, average='macro')
xgb_recall = recall_score(y_test, xgb_pred, average='macro')
xgb_f1 = f1_score(y_test, xgb_pred, average='macro')
print('XGBoost Accuracy:', xgb_acc)
print('XGBoost Precision:', xgb_precision)
print('XGBoost Recall:', xgb_recall)
print('XGBoost F1 Score:', xgb_f1)

# Train and test a Decision Tree model
dt = DecisionTreeClassifier()
dt.fit(X_train, y_train)
dt_pred = dt.predict(X_test)
dt_acc = accuracy_score(y_test, dt_pred)
dt_precision = precision_score(y_test, dt_pred, average='macro')
dt_recall = recall_score(y_test, dt_pred, average='macro')
dt_f1 = f1_score(y_test, dt_pred, average='macro')
print('Decision Tree Accuracy:', dt_acc)
print('Decision Tree Precision:', dt_precision)
print('Decision Tree Recall:', dt_recall)
print('Decision Tree F1 Score:', dt_f1)

# Train and test a Naive Bayes model
nb = GaussianNB()
nb.fit(X_train, y_train)
nb_pred = nb.predict(X_test)
nb_acc = accuracy_score(y_test, nb_pred)
nb_precision = precision_score(y_test, nb_pred, average='macro')
nb_recall = recall_score(y_test, nb_pred, average='macro')
nb_f1 = f1_score(y_test, nb_pred, average='macro')
print('Naive Bayes Accuracy:', nb_acc)
print('Naive Bayes Precision:', nb_precision)
print('Naive Bayes Recall:', nb_recall)
print('Naive Bayes F1 Score:', nb_f1)


Index(['expiration_id', 'src_port', 'dst_port', 'protocol',
       'bidirectional_duration_ms', 'bidirectional_packets',
       'bidirectional_bytes', 'src2dst_duration_ms', 'src2dst_packets',
       'src2dst_bytes', 'dst2src_last_seen_ms', 'bidirectional_stddev_ps',
       'bidirectional_max_ps', 'src2dst_stddev_ps', 'src2dst_max_ps',
       'bidirectional_stddev_piat_ms', 'src2dst_stddev_piat_ms',
       'application_confidence', 'splt_direction_2', 'splt_direction_3'],
      dtype='object')
Random Forest Accuracy: 0.9764957264957265
Random Forest Precision: 0.9395832945136986
Random Forest Recall: 0.9699265381083563
Random Forest F1 Score: 0.9539853097831467
XGBoost Accuracy: 0.9764957264957265
XGBoost Precision: 0.9391336332512803
XGBoost Recall: 0.9653803308795856
XGBoost F1 Score: 0.9517063644198034
Decision Tree Accuracy: 0.9636752136752137
Decision Tree Precision: 0.9247138278388278
Decision Tree Recall: 0.9129446476689398
Decision Tree F1 Score: 0.9180040610467021
Naive Bayes 

# export model II

In [13]:
import pickle
# Save the model as a pickle file
with open('/kaggle/working/phase2_model.pkl', 'wb') as f:
    pickle.dump(rf, f)

In [14]:

# Save the scaler
with open('standard_scaler_2.pkl', 'wb') as file:
    pickle.dump(scaler1, file)